In [36]:
import json

import polars as pl
from scipy.interpolate import CubicSpline
from jinja2 import Template

In [37]:
most_data = pl.read_csv("data/LandColor.csv")
forest_data = pl.read_csv("data/LandColorForest.csv")
data = most_data.join(forest_data.select("shapeName", "date", pl.selectors.starts_with("forest_")), on=["shapeName", "date"])

In [39]:
mapping = {"grassland": "Grasslands", "savanna": "Savannas", "wetland": "Wetlands", "forest": "Forests", "builtup": "Urban Areas", "water": "Water", "crops": "Croplands", "shrubs": "Shrubs"}
long = (data
    .select(pl.all().exclude([".geo", "system:index"]))
    .unpivot(index = ["date", "shapeGroup", "shapeName", "shapeType"], variable_name = "variable", value_name="meanReflectance")
    .with_columns(
        pl.from_epoch(pl.col("date"), time_unit="ms"),
        pl.col("variable").str.splitn('_',2).struct.rename_fields(["landCover","band"]),
    )
    .unnest("variable")
    .with_columns(
        pl.col.landCover.replace_strict(mapping),
        month = pl.col.date.dt.month()
    )
    .filter(shapeType="ADM0")
)

In [40]:
# Cubic spline from Blue Marble Next Gen
cs = CubicSpline([0,0.25,0.75,1], [0, 179, 240, 255])

In [41]:
means = long.group_by(["shapeGroup", "shapeName", "band", "landCover", "month"]).agg(pl.col.meanReflectance.mean()).drop_nans().with_columns(
                hex=pl.col.meanReflectance
                    .map_elements(lambda meanRefl: float(cs((meanRefl * 0.0001))), return_dtype=pl.Float32)
                    .cast(pl.UInt8, strict=False)
            ).drop_nulls()
means

shapeGroup,shapeName,band,landCover,month,meanReflectance,hex
str,str,str,str,i8,f64,u8
"""DMA""","""Dominica""","""green""","""Water""",4,193.904148,19
"""SLE""","""Sierra Leone""","""green""","""Grasslands""",6,529.462875,51
"""GRD""","""Grenada""","""blue""","""Water""",4,396.532168,39
"""YEM""","""Yemen""","""blue""","""Shrubs""",5,1006.415151,90
"""SWE""","""Sweden""","""green""","""Grasslands""",5,5919.533755,240
…,…,…,…,…,…,…
"""CMR""","""Cameroon""","""red""","""Grasslands""",2,1334.861616,114
"""GRC""","""Greece""","""green""","""Water""",12,393.857251,38
"""BDI""","""Burundi""","""green""","""Urban Areas""",10,1094.100422,97


In [42]:
def to_rgb(dataframe, groupby):
    groupby = set(groupby)
    later_select = groupby-{"band"}
    means = dataframe.group_by(groupby).agg(pl.col.meanReflectance.mean()).drop_nans()
    return (
        means
            .with_columns(
                hex=pl.col.meanReflectance
                    .map_elements(lambda meanRefl: float(cs((meanRefl * 0.0001))), return_dtype=pl.Float32)
                    .cast(pl.UInt8, strict=False)
            )
            .drop_nulls()
            .pivot(index=later_select, values="hex", on="band").select(*later_select, "red", "green", "blue")
    )

In [43]:
total_mean = to_rgb(long, ["shapeGroup", "shapeName", "band", "landCover"])
mean_per_month = to_rgb(long, ["shapeGroup", "shapeName", "band", "landCover", "month"])

In [44]:
monthly_mean_dicts = mean_per_month.to_dicts()
with open("src/colors.js", "w") as fp:
    fp.write("var colors = "+json.dumps(monthly_mean_dicts))

In [23]:
monthly_mean_dicts

[{'landCover': 'wetland',
  'month': 6,
  'shapeName': 'Luxembourg',
  'shapeGroup': 'LUX',
  'red': 46,
  'green': 57,
  'blue': 28},
 {'landCover': 'water',
  'month': 10,
  'shapeName': 'Poland',
  'shapeGroup': 'POL',
  'red': 31,
  'green': 38,
  'blue': 22},
 {'landCover': 'builtup',
  'month': 3,
  'shapeName': 'Lesotho',
  'shapeGroup': 'LSO',
  'red': 98,
  'green': 90,
  'blue': 53},
 {'landCover': 'wetland',
  'month': 5,
  'shapeName': 'Somalia',
  'shapeGroup': 'SOM',
  'red': 77,
  'green': 80,
  'blue': 48},
 {'landCover': 'shrubs',
  'month': 4,
  'shapeName': 'Honduras',
  'shapeGroup': 'HND',
  'red': 53,
  'green': 63,
  'blue': 30},
 {'landCover': 'grassland',
  'month': 4,
  'shapeName': 'Jordan',
  'shapeGroup': 'JOR',
  'red': 158,
  'green': 130,
  'blue': 84},
 {'landCover': 'grassland',
  'month': 4,
  'shapeName': 'Uruguay',
  'shapeGroup': 'URY',
  'red': 65,
  'green': 67,
  'blue': 33},
 {'landCover': 'shrubs',
  'month': 3,
  'shapeName': 'Bulgaria',
  's

In [8]:
countries = {}
for groupby, data in mean_per_month.sort("landCover").group_by("shapeName", "month"):
    country_name = groupby[0]  
    month = groupby[1]
    if country_name in countries:
        countries[country_name].update({
            month: [
                {
                    "landCover": row[-4],
                    "rgb": ",".join(map(str, row[-3:]))
                }
                for row in data.iter_rows()
            ]
        })
    else:
        countries[country_name] = {
            month: [
                {
                    "landCover": row[-4],
                    "rgb": ",".join(map(str, row[-3:]))
                }
                for row in data.iter_rows()
            ]
        }
countries = {k: dict(sorted(v.items())) for k,v in countries.items()}
    

In [9]:
countries

{'Georgia': {1: [{'landCover': 'GEO', 'rgb': '103,97,80'},
   {'landCover': 'GEO', 'rgb': '204,201,194'},
   {'landCover': 'GEO', 'rgb': '113,114,114'},
   {'landCover': 'GEO', 'rgb': '201,201,199'},
   {'landCover': 'GEO', 'rgb': '154,151,143'},
   {'landCover': 'GEO', 'rgb': '236,236,235'},
   {'landCover': 'GEO', 'rgb': '113,119,109'},
   {'landCover': 'GEO', 'rgb': '144,147,141'}],
  2: [{'landCover': 'GEO', 'rgb': '112,105,87'},
   {'landCover': 'GEO', 'rgb': '207,204,197'},
   {'landCover': 'GEO', 'rgb': '125,125,123'},
   {'landCover': 'GEO', 'rgb': '214,214,212'},
   {'landCover': 'GEO', 'rgb': '166,163,154'},
   {'landCover': 'GEO', 'rgb': '239,239,238'},
   {'landCover': 'GEO', 'rgb': '136,141,131'},
   {'landCover': 'GEO', 'rgb': '166,168,162'}],
  3: [{'landCover': 'GEO', 'rgb': '89,82,58'},
   {'landCover': 'GEO', 'rgb': '151,145,128'},
   {'landCover': 'GEO', 'rgb': '119,117,110'},
   {'landCover': 'GEO', 'rgb': '207,206,200'},
   {'landCover': 'GEO', 'rgb': '141,136,121'

In [10]:
reorganized_data = {}
for country, months in countries.items():
    reorganized_data[country] = {}
    for month, land_covers in months.items():
        for entry in land_covers:
            land_cover = entry['landCover']
            rgb = entry['rgb']
            if land_cover not in reorganized_data[country]:
                reorganized_data[country][land_cover] = {}
            reorganized_data[country][land_cover][month] = rgb

In [11]:
reorganized_data

{'Georgia': {'GEO': {1: '144,147,141',
   2: '166,168,162',
   3: '151,153,143',
   4: '105,109,91',
   5: '78,88,63',
   6: '66,78,50',
   7: '51,65,37',
   8: '47,58,32',
   9: '45,55,31',
   10: '51,58,36',
   11: '75,80,65',
   12: '117,120,112'}},
 'Zimbabwe': {'ZWE': {1: '51,59,30',
   2: '47,56,28',
   3: '43,51,26',
   4: '38,44,23',
   5: '37,40,21',
   6: '38,39,22',
   7: '42,40,23',
   8: '47,42,25',
   9: '55,47,30',
   10: '62,54,34',
   11: '63,58,34',
   12: '55,59,32'}},
 'Eritrea': {'ERI': {1: '84,86,45',
   2: '86,88,47',
   3: '92,94,52',
   4: '94,97,53',
   5: '90,93,50',
   6: '93,95,53',
   7: '100,98,59',
   8: '107,105,66',
   9: '99,98,55',
   10: '95,94,51',
   11: '89,89,47',
   12: '82,83,43'}},
 'Samoa': {'WSM': {1: '28,42,24',
   2: '31,46,26',
   3: '32,48,27',
   4: '28,44,24',
   5: '26,41,23',
   6: '25,40,21',
   7: '26,40,22',
   8: '28,42,23',
   9: '30,44,24',
   10: '34,48,27',
   11: '32,46,26',
   12: '29,44,25'}},
 'Tunisia': {'TUN': {1: '43,

In [37]:
html_template = """<!DOCTYPE html>
<html>
<head>
    <title>Land Use Color Palettes</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            padding: 0;
        }
        h1, h2, h3 {
            margin-top: 20px;
        }
        .palette {
            display: flex;
            flex-wrap: wrap;
            margin-bottom: px;
        }
        .color-block {
            width: 60px;
            height: 30px;
            margin: 0px;
            text-align: center;
            font-size: 10px;
            color: #000;
            display: flex;
            align-items: center;
            justify-content: center;
        }
.landuse {
    width: 100px;
    text-align: center;
    color: #000;
    display: flex;
    align-items: center;
    justify-content: right;
    margin-right: 10px;
}
    </style>
</head>
<body>
    <h1>Land Use Color Palettes</h1>
"""

def generate_palette_html(data):
    content = ""
    for country, land_uses in data.items():
        content += f"<h2>{country}</h2>\n"
        for land_use, months in land_uses.items():
            content += "<div class='palette'>\n"
            content += f"<div class='landuse'>{land_use.capitalize()}</div>\n"
            for month, rgb in months.items():
                rgb_tuple = tuple(map(int, rgb.split(',')))
                text_color = "#000" if sum(rgb_tuple) > 382 else "#fff"  # Choose text color based on brightness
                content += (f"<div class='color-block' style='background-color: rgb({rgb}); color: {text_color};'>"
                            f"</div>\n")
            content += "</div>\n"
    return content

html_content = generate_palette_html(reorganized_data)
final_html = html_template+html_content+"</body></html>"

# Save the HTML to a file
output_file = "land_use_palettes.html"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(final_html)

print(f"HTML file generated")


HTML file generated


In [44]:
# HTML Template for the palette page
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Country Color Palettes</title>
    <style>

        body {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
            color: #333;
        }
        header {
            background-color: #007acc;
            color: white;
            padding: 20px;
            text-align: center;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        h1 {
            margin: 0;
            font-size: 2.5em;
        }
        .container {
            max-width: 1200px;
            margin: 20px auto;
            padding: 20px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        }
        .country {
            margin-bottom: 30px;
            border-bottom: 1px solid #e0e0e0;
            padding-bottom: 15px;
        }
        .country h2 {
            font-size: 1.8em;
            margin-bottom: 10px;
            color: #007acc;
            border-left: 4px solid #007acc;
            padding-left: 10px;
        }
        .month {
            display: flex;
            align-items: center;
            margin-bottom: 0px;
        }
        .month-label {
            font-size: 1.2em;
            font-weight: bold;
            margin-right: 15px;
            width: 100px;
            color: #555;
        }
        .palette {
            display: flex;
            flex-wrap: wrap;
            gap: 8px;
        }
        .color {
            width: 100px;
            height: 50px;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 0.9em;
            font-weight: bold;
            text-transform: capitalize;
            color: white;
            #border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.15);
            transition: transform 0.2s ease-in-out;
        }
        .color:hover {
            transform: scale(1.1);
        }
        footer {
            text-align: center;
            padding: 20px;
            margin-top: 40px;
            background-color: #007acc;
            color: white;
            font-size: 0.9em;
        }
        .landcover-title {
            font-size: 1em;
            font-weight: bold;
            color: #666;
            margin-bottom: 5px;
        }
        .titles {
           display: flex;
        }
    </style>
</head>
<body>
    <header>
        <h1>Country Color Palettes</h1>
    </header>
    <div class="container">
        {% for country, landuse in countries.items() %}
        <div class="country">
            <h2>{{ country }}</h2>
            {% for landuse, months in landuse.items() %}
                <div class="month-label">{{ landuse }}</div>
                <div class="palette">
                {% for month, rgb in months.items() %}
                    <div class="color" style="background-color: rgb({{ rgb }});"></div>
                {% endfor %}
            {% endfor %}
            </div>
            </div>
            {% endfor %}
    </div>
    <footer>
        &copy; 2025 Country Color Palettes. All Rights Reserved.
    </footer>
</body>
</html>
"""

# Render the HTML
template = Template(html_template)
html_content = template.render(countries=reorganized_data)

# Save the HTML to a file
output_file = "land_cover_palettes.html"
with open(output_file, "w") as f:
    f.write(html_content)

print(f"HTML file has been generated and saved as {output_file}.")


HTML file has been generated and saved as land_cover_palettes.html.
